In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from easygui import *
import plotly.io as pio

In [3]:
#df is your players' data, just change the path to your data file's path.
df = pd.read_html(r"/Users/Zhaoyichun/Downloads/South.html",skiprows=0,)[0]
df.head()


,Name,Age,Personality,Best Pos,Expires,Transfer Value,Wage,Svt,Svp,Svh,...,Aer A/90,xG,Height,Cons,FA,Fls,Yel,Gwin,Clear,Cr A
0,Matteo Pessina,27,Spirited,M (C),30/6/2027,â¬93M - â¬114M,"â¬80,000 p/w",-,-,-,...,5.91,11.98,187 cm,16,79,21,1,76%,35.00,95
1,Tino Livramento,21,Fairly Determined,D (R),30/6/2027,â¬94M - â¬106M,"â¬62,000 p/w",-,-,-,...,9.80,0.61,186 cm,11,42,45,8,79%,186.00,349
2,Thiago Almada,23,Fairly Determined,AM (L),30/6/2026,â¬72M - â¬95M,"â¬35,000 p/w",-,-,-,...,3.11,10.50,171 cm,15,51,11,1,85%,9.00,112
3,GonÃ§alo InÃ¡cio,22,Fairly Professional,D (C),30/6/2028,â¬74M - â¬83M,"â¬58,000 p/w",-,-,-,...,13.07,0.71,187 cm,15,5,18,3,73%,299.00,5
4,JoÃ£o Cancelo,29,Balanced,D (R),30/6/2026,â¬62M - â¬68M,"â¬100,000 p/w",-,-,-,...,7.13,0.97,182 cm,13,28,58,14,74%,168.00,38


In [4]:
# Get rid of symbols or something not good for analyzing
pd.set_option('display.max_columns',40)
pd.set_option('display.max_rows',30)
for column in ['Svt', 'Svh', 'Svp', 'Ch C/90', 'DrbPG', 'Cr C', 'Cr C/A', 'Tck', 'Int/90', 'K Ps/90', 'Hdrs W/90',
               'Hdr %','Ps C/90', 'Pas %','Gls/90', 'Asts/90', 'ShT/90', 'Tck R', 'Gls', 'Shot %', 'Tcon/90',
               'Ps A/90', 'Mins', 'Dist/90', 'Av Rat', 'Ast', 'Tcon', 'Sv %', 'Shot/90', 'Aer A/90', 'xG', 'Height','Clear','Aer A/90'
              ]:
    df[column] = df[column].apply(str).str.replace('-','0')
    df[column] = df[column].apply(str).str.replace('%','')
    df[column] = df[column].apply(str).str.replace('cm','')
    df[column] = df[column].apply(str).str.replace('km','').astype(float)

    df['xG'] = df['xG'].apply(str).str.replace('0.0','0.01').astype(float)
    df['Wage'] = df['Wage'].apply(str).str.replace('p/w','')
    df['Wage'] = df['Wage'].apply(str).str.replace(',','')
    df['Wage'] = df['Wage'].apply(str).str.replace('â\x82¬','').astype(float)
    df['Transfer Value'] = df['Transfer Value'].apply(str).str.replace('â\x82¬','')
    df['Transfer Value'] = df['Transfer Value'].apply(str).str.replace('M','0')

/var/folders/js/l945ghm905b7hw9ky4qvhp0r0000gn/T/ipykernel_17946/2446072661.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  df['xG'] = df['xG'].apply(str).str.replace('0.0','0.01').astype(float)


In [5]:
# Define more types of stat
df['Tot_Sav'] = df['Svp'] + df['Svh'] + df['Svt']
df['Per_90'] = df['Mins'] / 90
df['Saves_90'] = df['Tot_Sav'] / df['Per_90']
df['Shots_Faced'] = df['Tcon'] + df['Tot_Sav']
df['Faced_90'] = df['Shots_Faced'] / df['Per_90']
df['Goal'] = df["Gls"] / df["xG"]
df['AttOutput'] = df['Gls/90'] + df['Asts/90']
df['ShTratio'] = df['ShT/90'] / df['Shot/90']
df['ShTratio'] = df['ShTratio'].fillna(0)
df['XG-Goal'] = df['Gls'] - df['xG'].astype(float)


In [6]:
#Analysing Goalkeepers
condition = (df['Best Pos']=='GK')
included = df[condition]
fig = px.scatter(included, x="Faced_90", y="Saves_90", text="Name", log_x=False, size_max=40, size='Sv %')
#fig.update_yaxes(autorange="reversed")
fig.update_traces(textposition='top center')
fig.update_layout(height=500)
fig.update_xaxes(title_text='Shots faced/90')
fig.update_yaxes(title_text='Saves made/90')
for template in ["seaborn"]:
    fig.update_layout(template=template, title="Shot Stopping")
fig.show()


In [7]:
#Analysing Defending Quality
fig = px.scatter(df, x="Tck", y="Hdrs W/90", text="Name", log_x=False, size_max=15, color="Ps C/90" , size='Tck R')
#fig.update_yaxes(autorange="reversed")
fig.update_traces(textposition='top center')
fig.update_layout(height=500)
fig.update_xaxes(title_text='Tackles Won/90')
fig.update_yaxes(title_text='Headers Won/90')
fig.add_hline(y=6, opacity=1, line_width=2, line_dash='dash', line_color='green')
fig.add_vline(x=0.94, opacity=1, line_width=2, line_dash='dash', line_color='green')
for template in ["seaborn"]:
    fig.update_layout(template=template, title="Defending Quality")
fig.show()

In [8]:
#Analysing Ground Duels ability
fig = px.scatter(df, x="Tck", y="Int/90", text="Name", log_x=False, size_max=20, color="Tck R", size= 'Fls')
#fig.update_yaxes(autorange="reversed")
fig.update_traces(textposition='top center')
fig.update_layout(height=500)
fig.update_xaxes(title_text='Tackles Won/90')
fig.update_yaxes(title_text='Interceptions/90')
fig.add_hline(y=1.77, opacity=1, line_width=2, line_dash='dash', line_color='green')
fig.add_hrect(y0=1.77, y1=df["Int/90"].max()+0.4, line_width=0, fillcolor="green", opacity=0.2)
for template in ["seaborn"]:
    fig.update_layout(template=template, title="Ground Duels")
fig.show()

In [9]:
#Analysing Aerial Duels ability
fig = px.scatter(df, x="Hdrs W/90", y="Hdr %", text="Name", log_x=False, size_max=60, color="Height")
#fig.update_yaxes(autorange="reversed")
fig.update_traces(textposition='top center')
fig.update_layout(height=500)
fig.update_xaxes(title_text='Headers Won/90')
fig.update_yaxes(title_text='Headers Won %')
fig.add_hline(y=80, opacity=1, line_width=2, line_dash='dash', line_color='green')
fig.add_hrect(y0=80, y1=100, line_width=0, fillcolor="green", opacity=0.2)
for template in ["seaborn"]:
    fig.update_layout(template=template, title="Aerial Duels")
fig.show()

In [10]:
#Analysing Ball Carrying ability
fig = px.scatter(df, x="Ch C/90", y="DrbPG", text="Name", log_x=False, size_max=30, color='FA')
#fig.update_yaxes(autorange="reversed")
fig.update_traces(textposition='top center')
fig.update_layout(height=500)
fig.update_xaxes(title_text='Progressive Passes/90')
fig.update_yaxes(title_text='Progressive Runs/90')
fig.add_hline(y=1.71, opacity=1, line_width=2, line_dash='dash', line_color='green')
fig.add_hrect(y0=1.71, y1=df["DrbPG"].max()+0.4, line_width=0, fillcolor="green", opacity=0.2)
for template in ["seaborn"]:
    fig.update_layout(template=template, title="Ball carrying skills")
fig.show()

In [11]:
#Analysing Crossing ability
fig = px.scatter(df, x="Cr C", y="Cr C/A", text="Name", log_x=False, size_max=20, color='Asts/90',size="Ast")
#fig.update_yaxes(autorange="reversed")
fig.update_traces(textposition='top center')
fig.update_layout(height=500)
fig.update_xaxes(title_text='Crossed Completed/90')
fig.update_yaxes(title_text='Crossing Accuracy')
for template in ["seaborn"]:
    fig.update_layout(template=template, title="Crossing ability")
fig.show()

In [12]:
#Analysing Chance Creation ability
fig = px.scatter(df, x="K Ps/90", y="Ch C/90", text="Name", log_x=False, size_max=20, color='Asts/90',size="Ast")
#fig.update_yaxes(autorange="reversed")
fig.update_traces(textposition='top center')
fig.update_layout(height=500)
fig.update_xaxes(title_text='Chance Creation/90')
fig.update_yaxes(title_text='Key Passes/90')
fig.add_vline(x=0.91, opacity=1, line_width=2, line_dash='dash', line_color='green')
fig.add_vrect(x0=0.91, x1=df["K Ps/90"].max()+0.4, line_width=0, fillcolor="green", opacity=0.2)
for template in ["seaborn"]:
    fig.update_layout(template=template, title="Chance Creation")
fig.show()

In [13]:
#Analysing Passing ability
fig = px.scatter(df, x="Ps C/90", y="Pas %", text="Name", log_x=False, size_max=20, color='K Ps/90',size= 'K Ps/90')
fig.update_traces(textposition='top center')
fig.update_layout(height=500)
fig.update_xaxes(title_text='Passes Completed/90')
fig.update_yaxes(title_text='Pass Completion %')
fig.add_hline(y=83, opacity=1, line_width=2, line_dash='dash', line_color='green')
fig.add_hrect(y0=83, y1=100, line_width=0, fillcolor="green", opacity=0.2)
for template in ["seaborn"]:
    fig.update_layout(template=template, title="Passing Ability")
fig.show()

In [14]:
#Analysing Attacking Impact for the team
fig = px.scatter(df, x="Gls/90", y="Asts/90", text="Name", log_x=False, size_max=20, color ='Av Rat', size= 'AttOutput')
#fig.update_yaxes(autorange="reversed")
fig.update_traces(textposition='top center')
fig.update_layout(height=500)
fig.update_xaxes(title_text='Goals/90')
fig.update_yaxes(title_text='Assists/90')
fig.add_hline(y=0.2, opacity=1, line_width=2, line_dash='dash', line_color='green')
fig.add_hrect(y0=0.2, y1=df["Asts/90"].max()+0.1, line_width=0, fillcolor="green", opacity=0.2)
for template in ["seaborn"]:
    fig.update_layout(template=template, title="Attacking Impact")
fig.show()

In [15]:
#Analysing Goalscoring Efficiency for the team
fig = px.scatter(df, x="Gls/90", y="ShT/90", text="Name", log_x=False, size_max=20, color='Shot/90', size = 'ShTratio')
#fig.update_yaxes(autorange="reversed")
fig.update_traces(textposition='top center')
fig.update_layout(height=500)
fig.update_xaxes(title_text='Goals/90')
fig.update_yaxes(title_text='Shots on Target/90')
fig.add_vline(x=0.39, opacity=1, line_width=2, line_dash='dash', line_color='green')
fig.add_vrect(x0=0.39, x1= df["Gls/90"].max() + 0.1, line_width=0, fillcolor="green", opacity=0.2)
for template in ["seaborn"]:
    fig.update_layout(template=template, title="Goalscoring Efficiency")
fig.show()

In [16]:
#See the relation between goals and xG
fig = px.scatter(df, x="Gls", y="xG", text="Name", log_x=False, size_max=15, color='ShT/90', size='Gls')
#fig.update_yaxes(autorange="reversed")
fig.update_traces(textposition='top center')
fig.update_layout(height=500)
fig.update_xaxes(title_text='Goals')
fig.update_yaxes(title_text='Expected Goals')

for template in ["seaborn"]:
    fig.update_layout(template=template, title="Goalscoring Expectation VS Goals")
fig.show()

In [17]:
#Analysing xG and Goals to see who outperform in the team
fig = px.scatter(df, x='XG-Goal', y="xG", text="Name", log_x=False, size_max=15, color='ShT/90', size='Gls')
#fig.update_yaxes(autorange="reversed")
fig.update_traces(textposition='top center')
fig.update_layout(height=500)
fig.update_xaxes(title_text='Actual Goals min xG')
fig.update_yaxes(title_text='Expected Goals')

for template in ["seaborn"]:
    fig.update_layout(template=template, title="Goals beyond expectation")
fig.show()

In [18]:
#Running distance per game and minutes played for each player
fig = px.scatter(df, x="Dist/90", y="Mins", text="Name", log_x=False, size_max=100, color='Av Rat')
#fig.update_yaxes(autorange="reversed")
fig.update_traces(textposition='top center')
fig.update_layout(height=500)
fig.update_xaxes(title_text='Distance covered/90')
fig.update_yaxes(title_text='Minitues')
for template in ["seaborn"]:
    fig.update_layout(template=template, title="Running Distance")
fig.show()

In [19]:
#Analysing players' rating to their hidden ability(consistency)

total = sum(df['Av Rat'])
avgRating = total / df.shape[0]

fig = px.scatter(df, x="Av Rat", y="Cons", text="Name", log_x=False, size_max=20, color='Av Rat',size = 'Cons')
#fig.update_yaxes(autorange="reversed")
fig.update_traces(textposition='top center')
fig.update_layout(height=500)
fig.update_xaxes(title_text='Average Rating')
fig.update_yaxes(title_text='Consistency')
fig.add_vline(x=avgRating, opacity=1, line_width=2, line_dash='dash', line_color='green')
fig.add_vrect(x0=avgRating, x1= df["Av Rat"].max() + 0.2, line_width=0, fillcolor="green", opacity=0.2)
for template in ["seaborn"]:
    fig.update_layout(template=template, title="Players' States")
fig.show()

In [20]:
#Analysing fouls and yellow cards

fig = px.scatter(df, x="Fls", y="Yel", text="Name", log_x=False, size_max=20, color='Av Rat',size = 'Cons')
#fig.update_yaxes(autorange="reversed")
fig.update_traces(textposition='top center')
fig.update_layout(height=500)
fig.update_xaxes(title_text='Fouls comitted')
fig.update_yaxes(title_text='Yellow cards')
for template in ["seaborn"]:
    fig.update_layout(template=template, title="Players' Diciplines of season")
fig.show()

In [21]:
#The relation between wage and performance 
fig = px.scatter(df, x="Wage", y="Av Rat", text="Name", log_x=False, size_max=20, color='Av Rat')
#fig.update_yaxes(autorange="reversed")
fig.update_traces(textposition='top center')
fig.update_layout(height=500)
fig.update_xaxes(title_text='Wage p/w')
fig.update_yaxes(title_text='Ave Rating')
for template in ["seaborn"]:
    fig.update_layout(template=template, title="No effort players ")
fig.show()

In [33]:
#The relation between player's transfer value and performance 

df['Average TransferValue'] = (df['Transfer Value'].str.split('-').str.get(0).astype(float) + df['Transfer Value'].str.split('-').str.get(1).astype(float)) / 2
df['Average TransferValue']

fig = px.scatter(df, x='Average TransferValue', y="Av Rat", text="Name", log_x=False, size_max=20, color='Av Rat')
#fig.update_yaxes(autorange="reversed")
fig.update_traces(textposition='top center')
fig.update_layout(height=500)
fig.update_xaxes(title_text='Transfer Value (M/€）')
fig.update_yaxes(title_text='Ave Rating')
for template in ["seaborn"]:
    fig.update_layout(template=template, title="Players' value")
fig.show()

In [23]:
#Players' imapct of possession on the pitch
# Goals Asists Tackles Int/90 Pass Pass rate
df['ImpactPos'] = 0.02*df['Ps C/90'] - 0.01*(df['Ps A/90']-df['Ps C/90']) + 0.3*df['Tck'] - 0.2*(df['Tck']/df['Tck R'] - df['Tck']) + 0.1*df['Hdrs W/90'] - 0.05*(df['Aer A/90'] - df['Hdrs W/90']) + 0.3*(df['FA']/df['Per_90']) - 0.1*(df['Fls']/df['Per_90']) + 5
df['ImpactPos'] = df['ImpactPos'].round(2)
impact = df[['Name','Best Pos','ImpactPos','Av Rat']]
condition = (impact['Best Pos']!='GK')
impact = impact[condition]
sorted_impact = impact.sort_values(by=['ImpactPos'], ascending=False)
sorted_impact.head(30).style.hide_index()\
                    .background_gradient(cmap='YlGn')

Name,Best Pos,ImpactPos,Av Rat
Benjamin Henrichs,D (R),8.540000,7.220000
Mohammed Salisu,D (C),8.220000,7.280000
Victor Nelsson,D (C),8.160000,7.290000
GonÃ§alo InÃ¡cio,D (C),8.140000,7.310000
Davide Frattesi,M (C),8.050000,7.700000
Tino Livramento,D (R),8.030000,7.480000
JoÃ£o Cancelo,D (R),7.840000,7.200000
Reinier,AM (C),7.760000,6.960000
Kyle Walker-Peters,D (R),7.730000,7.000000
Rodrigo Bentancur,M (C),7.640000,7.160000


In [24]:
#Players' imapct of attacking
df['ImpactAtt'] = 2*df['Gls/90'] - 0.2*(df['Shot/90']-df['ShT/90']) + 0.6*df['Asts/90'] + 0.25*df['K Ps/90'] + 0.15*(df['Cr C']/df['Per_90']) + 5
df['ImpactAtt'] = df['ImpactAtt'].round(2)
impacta = df[['Name','Best Pos','ImpactAtt','Av Rat']]
condition = (impacta['Best Pos']!='GK')
impacta = impacta[condition]
sorted_impacta = impacta.sort_values(by=['ImpactAtt'], ascending=False)
sorted_impacta.head(30).style.hide_index()\
                    .background_gradient(cmap='Reds')

Name,Best Pos,ImpactAtt,Av Rat
JuliÃ¡n Ãlvarez,AM (R),6.990000,7.180000
Thiago Almada,AM (L),6.760000,7.350000
James Ward-Prowse,M (C),6.710000,7.380000
Matteo Cancellieri,AM (R),6.670000,7.080000
Liam Delap,ST (C),6.320000,7.440000
Kiril Despodov,AM (L),6.270000,7.010000
Matteo Pessina,M (C),6.250000,7.310000
Davide Frattesi,M (C),6.220000,7.700000
Reinier,AM (C),6.170000,6.960000
Mikkel Damsgaard,AM (L),6.060000,6.950000


In [25]:
#Players' imapct of defending
df['ImpactDef'] = + 0.5*df['Tck'] - 0.2*(df['Tck']/df['Tck R'] - df['Tck']) + 0.3*df['Int/90'] + 0.1*df['Clear']/df['Per_90'] + 5
df['ImpactDef'] = df['ImpactDef'].round(2)
impactd = df[['Name','Best Pos','ImpactDef','Av Rat']]
condition = (impactd['Best Pos']!='GK')
impactd = impactd[condition]
sorted_impactd = impactd.sort_values(by=['ImpactDef'], ascending=False)
sorted_impactd.head(30).style.hide_index()\
                    .background_gradient(cmap='GnBu')

Name,Best Pos,ImpactDef,Av Rat
Benjamin Henrichs,D (R),8.290000,7.220000
Tino Livramento,D (R),8.100000,7.480000
JoÃ£o Cancelo,D (R),7.740000,7.200000
Mohammed Salisu,D (C),7.680000,7.280000
Kyle Walker-Peters,D (R),7.570000,7.000000
GonÃ§alo InÃ¡cio,D (C),7.500000,7.310000
Victor Nelsson,D (C),7.250000,7.290000
Kiril Despodov,AM (L),6.930000,7.010000
Davide Frattesi,M (C),6.840000,7.700000
Mikkel Damsgaard,AM (L),6.630000,6.950000


In [26]:
#Overall impact on the pitch
df['Impact'] =  (df['ImpactPos']+df['ImpactAtt']+df['ImpactDef']) / 3
impactall = df[['Name','Best Pos','Impact','ImpactPos','ImpactAtt','ImpactDef','Av Rat']]
condition = (impactall['Best Pos']!='GK')
impactall = impactall[condition]
sorted_impactall = impactall.sort_values(by=['Impact'], ascending=False)
sorted_impactall.head(30).style.hide_index()\
                    .background_gradient(cmap='Spectral')

Name,Best Pos,Impact,ImpactPos,ImpactAtt,ImpactDef,Av Rat
Benjamin Henrichs,D (R),7.370000,8.540000,5.280000,8.290000,7.220000
Tino Livramento,D (R),7.360000,8.030000,5.950000,8.100000,7.480000
Kyle Walker-Peters,D (R),7.046667,7.730000,5.840000,7.570000,7.000000
Davide Frattesi,M (C),7.036667,8.050000,6.220000,6.840000,7.700000
Mohammed Salisu,D (C),6.993333,8.220000,5.080000,7.680000,7.280000
JoÃ£o Cancelo,D (R),6.986667,7.840000,5.380000,7.740000,7.200000
GonÃ§alo InÃ¡cio,D (C),6.906667,8.140000,5.080000,7.500000,7.310000
JuliÃ¡n Ãlvarez,AM (R),6.873333,7.030000,6.990000,6.600000,7.180000
Victor Nelsson,D (C),6.846667,8.160000,5.130000,7.250000,7.290000
Reinier,AM (C),6.830000,7.760000,6.170000,6.560000,6.960000
